## Install pyTigerGraph

In [ ]:
# Setup
!pip install -U pyTigerGraph

## Add Imports and Establish Initial Connection

In [ ]:
# Imports
import json
import pandas as pd

from google.colab import drive
import pyTigerGraph as tg

drive.mount('/gdrive')

# Connection parameters
hostName = ''
userName = ''
password = ''
conn = tg.TigerGraphConnection(host=hostName, username=userName, password=password)

conn.graphname='TigyoreGraph'
secret = conn.createSecret()
authToken = conn.getToken(secret)
authToken = authToken[0]
conn = tg.TigerGraphConnection(host=hostName, graphname='TigyoreGraph', username=userName, password=password, apiToken=authToken)

print('Connected')

## Create Loading Jobs

## Subway Stations

### Clean Raw Data

In [ ]:
# Load Datasets
stop_times = pd.read_csv("/gdrive/My Drive/Tigyore/data/nyc/raw/stop_times.txt", sep=",")
stops = pd.read_csv("/gdrive/My Drive/Tigyore/data/nyc/raw/stops.txt", sep=",")
stop_seq = pd.merge(stop_times, stops, how='left', left_on='stop_id', right_on='stop_id')

# Get Route ID from Trip ID
stop_seq['route_id'] = stop_seq['trip_id'].str.split('\.\.')
stop_seq['route_id'] = stop_seq['route_id'].apply(lambda x: x[0])
stop_seq['route_id'] = stop_seq['route_id'].str.split('_')
stop_seq['route_id'] = stop_seq['route_id'].apply(lambda x: x[-1])

# Filter for Trips that have the longest sequence length for each Route
seq_len = stop_seq.groupby('trip_id').agg({'stop_sequence':lambda x: len(list(x))}).reset_index().rename(columns={'stop_sequence':'seq_len'})
max_seq_len_trips = pd.merge(stop_seq, seq_len, how='left', left_on='trip_id', right_on='trip_id')[['route_id', 'trip_id', 'seq_len']]
longest_trips = max_seq_len_trips.loc[max_seq_len_trips.reset_index().groupby(['route_id'])['seq_len'].idxmax()]['trip_id'].to_list()
print(longest_trips)
stop_seq = stop_seq[stop_seq['trip_id'].isin(longest_trips)]

# Save Datasets to csv
# stop_seq[['stop_id','stop_name','stop_lat', 'stop_lon']].drop_duplicates().to_csv("/gdrive/My Drive/Tigyore/data/nyc/nycSubwayStations.csv", index=False)
# stop_seq[['route_id']].drop_duplicates().to_csv("/gdrive/My Drive/Tigyore/data/nyc/nycSubwayServices.csv", index=False)
# stop_seq[['route_id', 'stop_id', 'stop_sequence', 'stop_name', 'stop_lat', 'stop_lon']].drop_duplicates().to_csv("/gdrive/My Drive/Tigyore/data/nyc/nycSubwayEdges.csv", index=False)

stop_seq[['trip_id', 'stop_id','stop_name','stop_lat', 'stop_lon', 'route_id', 'stop_sequence']].head(3)

### Create Load Data Job

Here it's important to note that the `$0`, `$1` values line up with the columns of your data.

In [ ]:
results = conn.gsql('''
USE GRAPH TigyoreGraph
SHOW VERTEX TrainStation
SHOW VERTEX TrainService
SHOW EDGE serve
''')

print(results)

In [ ]:
station_results = conn.gsql('''
  USE GRAPH TigyoreGraph
  BEGIN
  CREATE LOADING JOB load_subway_stations FOR GRAPH TigyoreGraph {
  DEFINE FILENAME MyDataSource;
  LOAD MyDataSource TO VERTEX TrainStation VALUES($0, $1, $2, $3) USING SEPARATOR=",", HEADER="true", EOL="\\n", QUOTE="double";
  }
  END
  ''')

service_results = conn.gsql('''
  USE GRAPH TigyoreGraph
  BEGIN
  CREATE LOADING JOB load_subway_services FOR GRAPH TigyoreGraph {
  DEFINE FILENAME MyDataSource;
  LOAD MyDataSource TO VERTEX TrainService VALUES($0) USING SEPARATOR=",", HEADER="true", EOL="\\n", QUOTE="double";
  }
  END
  ''')

edge_results = conn.gsql('''
  USE GRAPH TigyoreGraph
  BEGIN
  CREATE LOADING JOB load_subway_edges FOR GRAPH TigyoreGraph {
  DEFINE FILENAME MyDataSource;
  LOAD MyDataSource TO EDGE serve VALUES($0 TrainService, $1 TrainStation, $2) USING SEPARATOR=",", HEADER="true", EOL="\\n", QUOTE="double";
  }
  END
  ''')
print(station_results, service_results, edge_results)

In [ ]:
results = conn.gsql('''
USE GRAPH TigyoreGraph
SHOW JOB load_subway_stations
SHOW JOB load_subway_services
SHOW JOB load_subway_edges
''')

print(results)

### Load Data

In [ ]:
# Load the subway station file with the 'load_train_stations' job
data_file = '/gdrive/My Drive/Tigyore/data/nyc/nycSubwayStations.csv'
results = conn.uploadFile(data_file, fileTag='MyDataSource', jobName='load_subway_stations')
print(json.dumps(results, indent=2))

In [ ]:
# Load the subway service file with the 'load_subway_services' job
data_file = '/gdrive/My Drive/Tigyore/data/nyc/nycSubwayServices.csv'
results = conn.uploadFile(data_file, fileTag='MyDataSource', jobName='load_subway_services')
print(json.dumps(results, indent=2))

In [ ]:
# Load the subway edges file with the 'load_subway_edges' job
data_file = '/gdrive/My Drive/Tigyore/data/nyc/nycSubwayEdges.csv'
results = conn.uploadFile(data_file, fileTag='MyDataSource', jobName='load_subway_edges')
print(json.dumps(results, indent=2))